In [2]:
import pickle
import numpy as np
import xgboost as xgb

In [4]:
# Define median-based ensemble of surrogates
class medClassifier:
    def __init__(self, classifiers=None):
        self.classifiers = classifiers

    def predict(self, X):
        self.predictions_ = list()
        for classifier in self.classifiers:
            try:
                self.predictions_.append(classifier.predict(X)) #used for the random forest that is part of the ensemble
            except:
                X = xgb.DMatrix(X)
                self.predictions_.append(classifier.predict(X)) #used for the XGBoost models that are part of the ensemble
        med1 = np.median(self.predictions_, axis=0) #median of predictions
        mean1 = np.mean(self.predictions_, axis=0) #mean of predictions
        noise_param = 1 #amount of noise, set to 0 for deterministic function
        out = med1 + noise_param*np.random.rand()*np.abs(med1-mean1) #add more noise if median is far from mean, indicating more uncertainty, also all noise is positive to focus on minimizing parts with more certainty
        return out

In [6]:
# Load Ensemble
#Ensemblefile = os.path.join(folder_path,"Ensemble.pkl")
Ensemblefile = "Ensemble.pkl"
with open(Ensemblefile, 'rb') as file:
    Ensemble = pickle.load(file)


#This is the first objective function
def evaluate_WWensemble(x):
    x = np.array([x])
    pred = Ensemble.predict(x)
    return pred[0]

In [7]:
# Try the function
import time

x1 = [0.16583492632257624, 0.4871309875290023, 0.24153605985017246, 0.2954912222384124, 0.9558389075666868, 0.7992480932223422, 0.5400992985215289, 0.14902261675540462, 0.7592757901802544, 0.9983162571623986]

t0 = time.time()

y1 = evaluate_WWensemble(x1)

t1 = time.time()



print(y1)

print('time', t1-t0)

-8.8238325
time 0.05425715446472168


In [9]:
# Try another point
x2 = [0.4482183477205983, 0.027409116218383683, 0.25874372878562424, 0.6394764496282036, 1.0, 0.0, 1.0, 0.4692043394584843, 0.566542139621032, 0.0]

t0 = time.time()

y2 = evaluate_WWensemble(x2)

t1 = time.time()

print(y2)

print('time', t1-t0)


-58.789932
time 0.15554356575012207


In [10]:
from scipy.stats import truncnorm

# Build second objective
rotor_diameter = 126 # in meters
farm_length = 333.33*5 # in meters

def objective2(x):

    x = np.array([x])
    coords = np.resize(x,(2,5)) # wind turbine coordinates

    # Use a Monte Carlo simulation for the birds, who all fly from top to bottom at an x-location with a normal distribution. The mean of the normal distribution is far to the left of the wind farm.
    bird_mean = -25000 #width of bird corridor is 50km
    x_sigma = 4 # assume this many sigma of birds to stay within the planned corridor
    bird_std = (25000/x_sigma)
    #simulate birds (location in meters)
    nr_birds = 1000 # number of birds
    birds = truncnorm.rvs(x_sigma,x_sigma+farm_length/bird_std,loc=bird_mean,scale=bird_std,size=nr_birds) # Uses a truncated normal distribution, only sampling in the wind farm, not the entire bird corridor

    #check how many birds are close to a wind turbine (everything right of the leftmost turbine - rotor_diameter is dangerous area)
    leftmost = np.min(coords[0]) #location of leftmost turbine, in [0,1] units
    leftmost = leftmost*farm_length #change to meters
    threshold = leftmost-rotor_diameter #threshold of where the dangerous area starts (from left to right)

    close_birds = np.sum(birds >= threshold)/nr_birds #check how many birds fly to the right of the threshold
    return close_birds


In [18]:
# Try the second objective function

t0 = time.time()

y1_2 = objective2(x1)

t1 = time.time()



print(y1_2)

print('time', t1-t0)

0.872
time 0.002704620361328125


In [27]:
# Try the second objective function

t0 = time.time()

y2_2 = objective2(x2)

t1 = time.time()



print(y2_2)

print('time', t1-t0)

1.0
time 0.002926349639892578


In [28]:
# Build the constraint

from scipy.spatial.distance import cdist

def constraint1(x):
    coords = np.resize(x,(5,2))
    min_dist = 999999 #minimum distance between turbines (Euclidean)

    for turb in range(4):
        dists = cdist([coords[turb]],coords[turb+1:])
        next_min = np.min(dists)
        if next_min < min_dist:
            min_dist = next_min

    if min_dist*farm_length < 2*rotor_diameter:
        constr = 0 # constraint not satisfied, wind turbines are too close to each other
    else:
        constr = 1 # constraint satisfied
    return constr

In [36]:
# Try the constraint function

t0 = time.time()

c1 = constraint1(x1)

t1 = time.time()



print(c1)

print('time', t1-t0)

1
time 0.00035381317138671875


In [40]:
# Try the constraint function

t0 = time.time()

c2 = constraint1(x2)

t1 = time.time()



print(c2)

print('time', t1-t0)

0
time 0.0008757114410400391
